# change point detection validation


1. First randomly select from the 280 tiles, then within that tile randomly select a pixel
2. If that pixel is all null, discard and repeat from step 1, do not increment pixel counter
3. If that pixel is not null, plot the fillna version and the smoothed version,
4. Increment pixel counter for with/without break point
5. Repeat until we get 100 points for each


In [ ]:
%load_ext autoreload
%autoreload 2

import xarray as xr
from matplotlib import cm
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from carbonplan_styles.mpl import set_theme
set_theme()
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
import numpy as np
import seaborn as sns
import pandas as pd
from carbonplan_trace.tiles import tiles
from carbonplan_trace.v0.core import coarsen_emissions
from carbonplan_data.utils import set_zarr_encoding

In [ ]:
import random
from carbonplan_trace.v1.biomass_rollup import open_biomass_tile

random.seed(1)

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def pick_tile_and_pixel(version):
    done = False
    while not done:
        tile_id = random.choice(tiles)
        ds = open_biomass_tile(tile_id, version)
        i = random.randint(0, len(ds.lat) - 1)
        j = random.randint(0, len(ds.lon) - 1)
        pixel = ds.isel(lat=i, lon=j)
        if pixel.AGB.sum().values > 0:
            done = True

    return pixel.load()


def plot_pixel(pixel, y0=2014):
    n = len(pixel.AGB)
    xs = np.arange(y0, y0 + n)
    plt.figure()
    plt.plot(xs, pixel.AGB.values, label="smooth")
    plt.plot(xs, pixel.AGB_na_filled.values, label="na_filled")
    plt.plot(xs, pixel.AGB_raw.values, label="raw")
    plt.legend()
    lat = pixel.lat.values
    lon = pixel.lon.values
    breakpoint = pixel.breakpoint.values
    pvalue = np.round(pixel.pvalue.values, 3)
    plt.title(f"({lat},{lon}) breakpoint={breakpoint}, pvalue={pvalue:.3f}")
    plt.show()
    plt.close()
    return {"lat": lat, "lon": lon, "breakpoint": breakpoint, "pvalue": pvalue}

In [ ]:
with_break = 0
no_break = 0
no_model = 0
n = 50
out = []

while with_break < n or no_break < n or no_model < n:
    pixel = pick_tile_and_pixel(version)
    breakpoint = pixel.breakpoint.values
    pvalue = pixel.pvalue.values
    # has a break point and p value <= 0.05
    if (not np.isnan(breakpoint)) and (pvalue <= 0.05):
        if with_break < n:
            out.append(plot_pixel(pixel))
        with_break += 1

    # no break point but p value <= 0.05
    elif (np.isnan(breakpoint)) and (pvalue <= 0.05):
        if no_break < n:
            out.append(plot_pixel(pixel))
        no_break += 1

    # no break point and p value > 0.05
    elif (np.isnan(breakpoint)) and (pvalue > 0.05):
        if no_model < n:
            out.append(plot_pixel(pixel))
        no_model += 1

In [ ]:
pd.DataFrame(out).to_csv("change_point_detection_eval.csv", index=False)